In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [ ]:
df=pd.read_csv('AAPL.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df1 = df.reset_index()['Low']

In [ ]:
df1

In [ ]:
type(df1)

In [ ]:
np.where(pd.isnull(df1))

In [ ]:
df1 = df1.dropna()

In [ ]:
np.where(pd.isnull(df1))

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(df1)

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [ ]:
df1

In [ ]:
training_size=int(len(df1)*0.80)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

In [ ]:
training_size, test_size

In [ ]:
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0] 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

In [ ]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [ ]:
model=Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam', metrics=['mean_squared_error'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,epochs=100, batch_size=64, shuffle=False)

In [ ]:
test_predict=model.predict(X_test)

In [ ]:
print(f'MSE {mean_squared_error(y_test, test_predict)}')
print(f'R2 {r2_score(y_test, test_predict)}')

In [ ]:
test_predict=scaler.inverse_transform(test_predict)
train_predict=model.predict(X_train)
train_predict=scaler.inverse_transform(train_predict)

In [ ]:
look_back=100
trainPredictPlot = np.empty_like(df1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict

testPredictPlot = np.empty_like(df1)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1, :] = test_predict

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(scaler.inverse_transform(df1),label='Main dataset')
plt.plot(trainPredictPlot,label='Train dataset')
plt.plot(testPredictPlot,label='Test dataset')
plt.legend()

In [ ]:
x_input=df1[len(df1)-100:].reshape(1,-1)

In [ ]:
x_input = x_input.reshape((1, 100,1))
y_predict = model.predict(x_input, verbose=0)
next_day_value = scaler.inverse_transform(y_predict)

In [ ]:
next_day_value

Next day - low price - 124.88(USD)